## SDP with group structures: A Demo

In [1]:
import numpy as np
import cvxpy as cvx

(CVXPY) Jul 18 12:19:22 PM: Encountered unexpected exception importing solver SCS:
ImportError('dlopen(/Users/yibin/.local/lib/python3.7/site-packages/scs-3.2.0-py3.7-macosx-10.9-x86_64.egg/_scs_direct.cpython-37m-darwin.so, 2): Symbol not found: _aligned_alloc\n  Referenced from: /Users/yibin/.local/lib/python3.7/site-packages/scs-3.2.0-py3.7-macosx-10.9-x86_64.egg/scs/.dylibs/libgomp.1.dylib (which was built for Mac OS X 10.15)\n  Expected in: /usr/lib/libSystem.B.dylib\n in /Users/yibin/.local/lib/python3.7/site-packages/scs-3.2.0-py3.7-macosx-10.9-x86_64.egg/scs/.dylibs/libgomp.1.dylib')


Get the data

In [2]:
p = 100
corrMatrix = np.random.uniform(size=(p,p))
cat_name_to_idx = {'BsmtFinType2': [36, 37, 38, 39, 40], 'HeatingQC': [41, 42], 
'Neighborhood': [43, 44, 45, 46], 'SaleType': [47, 48, 49, 50], 
'Condition2': [51, 52], 'GarageFinish': [53, 54, 55, 56, 57], 
'LandContour': [58, 59, 60], 
'BsmtFinType1': [61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]}

In [3]:
# build a more conveninent data structure to encode group size information
# group_sizes contains the number of dummy variables in each group. If a variable is numerical, then its group size = 1
group_sizes = []
for i in range(p):
    # a flag that indicates if a variable is numerical or categorical
    num = True
    for group_indices in cat_name_to_idx.values():
        if i == group_indices[0]:
            num = False
            group_sizes.append(len(group_indices))
            break
        elif i in group_indices:
            num = False
            break
    # numerical variables
    if num:
        group_sizes.append(1)
print(f'Number of variables in SDP: {len(group_sizes)}')

Number of variables in SDP: 58


Get the block-diagonal correlation matrix

In [4]:
# define a mask matrix that helps us only keep the block-diagonal entries
mask_mat = np.zeros((p,p))
i = 0
for g_size in group_sizes:
    mask_mat[i:i+g_size,i:i+g_size] = np.ones((g_size,g_size))
    i += g_size
block_corr = corrMatrix * mask_mat
block_corr[35:40,35:40]

array([[0.77648864, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.28971091, 0.0454747 , 0.83685529, 0.31214709],
       [0.        , 0.58308317, 0.21929226, 0.75173261, 0.55653359],
       [0.        , 0.41725417, 0.26926378, 0.83621019, 0.91624361],
       [0.        , 0.88198946, 0.84179662, 0.67995362, 0.93595746]])

### Set up the SDP problem

\begin{align*}
    \min_{\gamma_j}&\, \sum_{j=1}^m\,(1-\gamma_j)\,\|\Sigma_{G_j,G_j}\|_F \\
    \text{s.t.}&\, 0\leq \gamma_j \leq 1\hspace{0.2cm}\forall\,j, \\
    &\,S \preceq 2\Sigma
\end{align*}

In [5]:
m = len(group_sizes) # number of groups
gamma = cvx.Variable(m)

Challenge: We want to repeat each $\gamma_j$ for $|G_j|$ times to build diag $\{\gamma_1\cdot I_{G_1,G_1},\dots, \gamma_m\cdot I_{G_m,G_m}\}$ (this is `gamma_mat` in my code). I seemed to mess up NumPy objects and cvxpy objects, which are symbolic expressions. How can we do this step?

An attempt to aviod *assigning numbers to part of a matrix* was that I wanted to use Kronecker product to represent diag $\{\gamma_1\cdot I_{G_1,G_1},\dots, \gamma_m\cdot I_{G_m,G_m}\}$, but the group sizes $|G_j|$ are different over all $j$'s. Thus, we cannot use Kronecker product.

In [8]:
# repeat each gamma_j by the number of dummy variables (a dictionary is not iterable by index)
# should gamma_mat be a numpy array or cvxpy symbolic matrix?
# Attempt 1
gamma_mat = np.eye(p)
i = 0
for j in range(m):
    g_size = group_sizes[j]
    gamma_mat[i:i+g_size,i:i+g_size] *= gamma[j]
    i += g_size
S = gamma_mat @ block_corr

# Attempt 2
gamma_mat = cvx.Variable((p,p))
i = 0
for j in range(m):
    g_size = group_sizes[j]
    gamma_mat[i:i+g_size,i:i+g_size] = gamma[j] * np.eye(g_size)
    i += g_size
S = gamma_mat @ block_corr

RuntimeError: 
You're calling a NumPy function on a CVXPY expression. This is prone to causing
errors or code that doesn't behave as expected. Consider using one of the
functions documented here: https://www.cvxpy.org/tutorial/functions/index.html


Set the objectives and constraints

In [ ]:
# use a for loop to set up the objective
# Note that if we use Frobenius norm SQUARED, then we don't need to use a for loop. 
# The objective then is the Frobenius norm of block_corr - S.
objective = 0
i = 0
for j in range(len(group_sizes)):
    g_size = group_sizes[j]
    objective += (1-gamma[j]) * cvx.norm(block_corr[i:i+g_size,i:i+g_size],'fro')
    i += g_size

constraints = [0 <= gamma, gamma <= 1, S << 2*corrMatrix]

Solve the problem

In [ ]:
prob = cvx.Problem(objective,constraints)
prob.solve()

print(f'Solution: {gamma.value}')